In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib import animation
from collections import Counter


In [ ]:
def adjust_map_coordinates(corordinates):
    y_cordinate = []
    midpoint = 982/2
    x_length = 982
    y_length = 982
    for j in list(corordinates['y_coordinate']):
        if j > midpoint:
            y_cordinate.append(y_length-j)
        elif j <midpoint:
            y_cordinate.append(x_length-j)
        else:
            y_cordinate.append(j)
    corordinates['y_coordinate'] = y_cordinate
    pos = {}
    for key,x,y in zip(list(corordinates['gate_name']),list(corordinates['x_coordinate']),list(corordinates['y_coordinate'])):
        pos[key] = np.array([x,y])
    return corordinates, pos

# cleaning

In [ ]:
import json

In [ ]:
file_ = open("/content/sample_data/json_data.json")

In [ ]:
data = json.load(file_)

In [ ]:
nodes_df = pd.DataFrame(columns=["in_date","out_date","node","edge","count"])

In [ ]:
for cartype in data['data']:
  for car in data['data'][cartype]:
    start = pd.to_datetime(car['enter_date'])
    for path in car['path_taken']:
      end = start + pd.Timedelta( minutes = list(path.values())[0])
      temp = list(path.keys())[0]
      nodes_df.loc[len(nodes_df)] = [str(start),str(end),temp.split(" ")[0],temp,1]
      start = end

  print(cartype+"  Completed")


1  Completed
2  Completed
3  Completed
4  Completed
5  Completed
6  Completed
2P  Completed


In [ ]:
nodes_df['in_date'] = pd.to_datetime(nodes_df['in_date'])
nodes_df['out_date'] = pd.to_datetime(nodes_df['out_date'])

In [ ]:
nodesdf = nodes_df.sort_values(by="in_date")

In [ ]:
nodesdf.to_csv("nodes.csv")

In [ ]:
nodesdf.reset_index(inplace=True,drop=True)

# Test

In [ ]:
start = list(nodesdf['in_date'])[0] -pd.Timedelta(seconds=5)
now = start +pd.Timedelta(minutes=1050)
temp = nodesdf[(nodesdf['in_date']<now) & (nodesdf['out_date']>now)]
nodes, n_weights = tuple(map(list,(Counter(temp['node']).keys() ,Counter(temp['node']).values())))
edges, e_weights =  tuple(map(list,(Counter(temp['edge']).keys(),Counter(temp['edge']).values())))
edges = [(i.split(" ")[0],i.split(" ")[-1]) for i in edges]

In [ ]:
c = max(nodes_df['out_date']) - min(nodes_df['in_date'])

In [ ]:
x  =[i.total_seconds() for i in list(nodes_df['out_date']-nodes_df['in_date'])]

In [ ]:
corordinates = pd.read_csv('/content/sample_data/Mapping Park Area.csv')
corordinates,pos = adjust_map_coordinates(corordinates)

In [ ]:
temp_g = nx.DiGraph()
start = list(nodesdf['in_date'])[0] -pd.Timedelta(seconds=5)
def update(frame):
    now_in = start +pd.Timedelta(minutes=(frame+1)*120)
    in_date = nodesdf['in_date']
    now_minus_in = now_in - pd.Timedelta(minutes=120)
    now_out = start +pd.Timedelta(minutes=(frame)*120)
    now_minus_out = now_out -pd.Timedelta(minutes=120)
    out_date = nodesdf['out_date']
    in_ = (in_date>=now_minus_in) & (in_date<=now_in)
    out_ = (out_date>now_minus_out) & (out_date<now_out)
    in_df = nodesdf[in_]
    out_df = nodesdf[out_]
    in_nodes,in_node_weights = tuple(map(list,(Counter(in_df['node']).keys() ,Counter(in_df['node']).values())))
    out_nodes,out_node_weights = tuple(map(list,(Counter(out_df['node']).keys() ,Counter(out_df['node']).values())))

    in_edges, in_edge_weights =  tuple(map(list,(Counter(in_df['edge']).keys(),Counter(in_df['edge']).values())))
    out_edges, out_edge_weights =  tuple(map(list,(Counter(out_df['edge']).keys(),Counter(out_df['edge']).values())))


    in_edges = [(i.split(" ")[0],i.split(" ")[-1]) for i in in_edges]
    out_edges = [(i.split(" ")[0],i.split(" ")[-1]) for i in out_edges]

    g_nodes = list(temp_g.nodes)
    g_edges = list(temp_g.edges)
    nodes_add = []
    nodes_weigths = []
    for k,j in zip(in_nodes,in_node_weights):
      if k not in g_nodes:
        nodes_add.append(k)
        nodes_weigths.append(j)
    edges_add = []
    edge_add_weights =[]
    for k,j in zip(in_edges,in_edge_weights):
      if k not in g_edges:
        edges_add.append(k)
        edge_add_weights.append(j)

    edges_rem = []
    edge_rem_weights =[]
    for k,j in zip(out_edges,out_edge_weights):
      if k in g_edges:
        edges_rem.append(k)
        edge_rem_weights.append(j)
    temp_g.add_nodes_from(nodes_add)
    temp_g.add_edges_from(edges_add)
    temp_g.remove_edges_from(edges_rem)
    temp_g.remove_nodes_from(list(nx.isolates(temp_g)))
    temp_g.remove_edges_from(nx.selfloop_edges(temp_g))
      # print(G.nodes)
    # print(np.array(n_weights))
    ax.clear()
    nx.draw_networkx_nodes(temp_g, pos,nodelist=nodes_add,node_size=np.array(nodes_weigths)*1500,ax=ax)
    nx.draw_networkx_labels(temp_g, pos,font_size=28,ax= ax)
    nx.draw_networkx_edges(temp_g, pos,edgelist=edges_add,width=np.array(edge_add_weights)*5,ax=ax, arrowsize=30)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    textstr = "Date : " + str(now_in.date()) + " \n  Time: "+str(now_in.time())
    ax.text(0, 1, textstr, transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props,color="r")

fig, ax = plt.subplots(figsize=(30,20))
ani = FuncAnimation(fig, update, frames=47635,interval=1000)
writervideo = animation.FFMpegWriter(fps=5)
ani.save('/content/drive/MyDrive/final_1.mp4', writer=writervideo)
plt.close()
